## Twitter Classification - Republican Democrats - Collecting the Data

In [1]:
import pandas as pd
import twint
import nest_asyncio
nest_asyncio.apply()
import json


The first thing I did in this project is to prepare in different file a list of all the Twitter accounts of congress members (House + Senate). For preparing the list I used the Data from the website http://www.tweetcongress.org/. I saved the data in two Json forms, one for each party (independent members were counts as Democrats).

## ________________________________

Then I created a function using the library Twint in order to scrape all the tweets of the congress members since 1.1.2019 and store it on Json files

In [23]:
def twitter_create_df(members,name):
    
    try:
    
        for t in members:
    
            c = twint.Config()
            c.Username = t
            c.Since = "2019-01-01"
            c.Store_json = True
            c.Format = "{username}"
            c.Custom["tweet"] = ["username","tweet"]
    # Name of the directory
            c.Output = str(name)
            twint.run.Search(c)
    except:
        pass
        
    df = pd.read_json(name+'/tweets.json', lines=True)
    df1 =  df.groupby('username')['tweet'].apply(' '.join).reset_index()
        
    return df1

#### Importing the lists with the twitter accounts and apply the function

In [2]:
with open('Rep.json', 'r') as f:
        Rep = json.load(f)

In [1]:
Rep = twitter_create_df(rep1,'Republican')

In [3]:
Rep[125:130]

['repdonbacon',
 'MarkAmodeiNV2',
 'RepChrisSmith',
 'RepPeteKing',
 'RepChrisCollins']

In [34]:
rep1 = Rep[126:]

In [1]:
Rep_df1 = twitter_create_df(rep1,'Republican1')

In [5]:
Rebuplican1 = pd.read_json('Republican/tweets.json', lines=True)
Republican2 = pd.read_json('Republican1/tweets.json', lines=True)
Final_Rep = pd.concat([Rebuplican1, Republican2], axis=0, ignore_index=True)

In [6]:
Final_Rep.head()

,tweet,username
0,I’m so sorry to hear this news about Rod and h...,robert_aderholt
1,My family and I are praying for the Bramblett ...,robert_aderholt
2,Proudly built on American soil! ULA built #At...,robert_aderholt
3,Redstone Arsenal being named to the short list...,robert_aderholt
4,It's National Travel & Tourism Week and there ...,robert_aderholt


In [13]:
Final_Rep['Party'] = 'R'

In [14]:
Final_Rep.head(3)

,tweet,username,Party
0,I’m so sorry to hear this news about Rod and h...,robert_aderholt,R
1,My family and I are praying for the Bramblett ...,robert_aderholt,R
2,Proudly built on American soil! ULA built #At...,robert_aderholt,R


In [15]:
Final_Rep.to_csv('Final_Rep.csv')

#### The same process to the Democrats

In [4]:
with open('Dems.json', 'r') as f:
        Dems = json.load(f)

In [1]:
twitter_create_df(Dems,'Democrats1')

In [7]:
Democrats = pd.read_json('Democrats/tweets.json', lines=True)

In [8]:
Democrats["Party"] = 'D'
Democrats.head()

,tweet,username,Party
0,.@repjohnlewis was gracious enough to spend ti...,repterrisewell,D
1,"At 450 pages, the Mueller Report is lengthy an...",repterrisewell,D
2,"I am so grateful to have Samuel, our Selma int...",repterrisewell,D
3,Anyone who knows me knows my dad was an instru...,repterrisewell,D
4,Please help me welcome our newest summer inter...,repterrisewell,D


In [9]:
Democrats= Democrats[Democrats.username != "ilhan"]

In [10]:
Ilhan = pd.read_json('ilhanMN/tweets.json', lines=True)
Ilhan['Party'] = 'D'
Ilhan.head(2)

,tweet,username,Party
0,One word: Impeachment. https://twitter.com/abc...,ilhanmn,D
1,Children in our country shouldn’t go into debt...,ilhanmn,D


In [11]:
Democrats_final = pd.concat([Democrats, Ilhan], axis=0, ignore_index=True)

In [12]:
Democrats_final.to_csv('Democrats_final.csv')

#### Creating joint data file for Democrats and Republicans

In [17]:
Final_Congress_tweets = pd.concat([Democrats_final, Final_Rep], axis=0, ignore_index=True)

In [18]:
len(Final_Congress_tweets)

176280

In [19]:
Final_Congress_tweets.to_csv(r'Final_Congress.csv')

## EDA

In [ ]:
import nltk
from nltk.collocations import *
from nltk.stem import PorterStemmer
from nltk.util import ngrams

In [100]:
Final_Congress_tweets  = pd.read_csv('Final_Congress.csv')
Final_Congress_tweets.drop(['Unnamed: 0'], axis=1, inplace  = True)
Final_Congress_tweets.head()

,tweet,username,Party
0,.@repjohnlewis was gracious enough to spend ti...,repterrisewell,D
1,"At 450 pages, the Mueller Report is lengthy an...",repterrisewell,D
2,"I am so grateful to have Samuel, our Selma int...",repterrisewell,D
3,Anyone who knows me knows my dad was an instru...,repterrisewell,D
4,Please help me welcome our newest summer inter...,repterrisewell,D


In [101]:
Final_Congress_tweets.groupby('username')['tweet'].count().sort_values(ascending=False).head()

username
pattymurray      1531
repdmp           1362
repjayapal       1314
senrobportman    1274
senwhitehouse    1261
Name: tweet, dtype: int64

In [103]:
Final_Rep.groupby('username')['tweet'].count().sort_values(ascending=False).head()

username
senrobportman    1274
senrickscott     1150
johncornyn       1107
senrubiopress    1026
lisamurkowski     981
Name: tweet, dtype: int64

In [102]:
Final_Congress_tweets.groupby('Party')['tweet'].count().sort_values(ascending=False).head()


Party
D    112044
R     64236
Name: tweet, dtype: int64

In [104]:
Final_Congress_tweets['tweet']  = Final_Congress_tweets['tweet'].apply(lambda x: x.lower())

In [105]:
Final_Congress_tweets.head(3)

,tweet,username,Party
0,.@repjohnlewis was gracious enough to spend ti...,repterrisewell,D
1,"at 450 pages, the mueller report is lengthy an...",repterrisewell,D
2,"i am so grateful to have samuel, our selma int...",repterrisewell,D


#### Joining to tweets by parties and tokenization

In [106]:
Parties_Tweet  =Final_Congress_tweets.groupby('Party')['tweet'].apply(' '.join).reset_index()

Parties_Tweet

,Party,tweet
0,D,.@repjohnlewis was gracious enough to spend ti...
1,R,i’m so sorry to hear this news about rod and h...


In [107]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
Parties_Tweet["tweet"] = Parties_Tweet['tweet'].apply(lambda x: nltk.regexp_tokenize(x,pattern))

In [ ]:
Parties_Tweet['tweet'][0] = [word for word in Parties_Tweet['tweet'][0] if word in vocab_total_D_R]
Parties_Tweet['tweet'][1] = [word for word in Parties_Tweet['tweet'][1] if word in vocab_total_D_R] 

In [ ]:
import string
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += ['0','1','2','3','4','5','6','7','8','9','com','https','www',
                   'pic','twitter','ly','http','html','gov','cfm','utm']
stopwords_list += list(string.ascii_lowercase)


Parties_Tweet['tweet'][0] = [word for word in Parties_Tweet['tweet'][0] if word not in stopwords_list ] 
Parties_Tweet['tweet'][1] = [word for word in Parties_Tweet['tweet'][1] if word not in stopwords_list ] 

In [27]:
Parties_Tweet.head()

,Party,tweet
0,D,"[repjohnlewis, gracious, enough, spend, time, ..."
1,R,"[sorry, hear, news, rod, wife, ability, radio,..."


In [28]:
print (len(Parties_Tweet['tweet'][0]), len(Parties_Tweet['tweet'][1]))

2415510 1329313


In [29]:
vocab1 = list(set(Parties_Tweet['tweet'][0]))
vocab2 = list(set(Parties_Tweet['tweet'][1]))
vocab_total =list(set(vocab1+vocab2))
print (len(vocab1), len(vocab2))
print (len(vocab_total))


122462 92327
181194


#### Ngrams and word frequencies

In [30]:
D_congress_freqdist =  nltk.FreqDist(Parties_Tweet["tweet"][0])

In [31]:
#showing democrats most common word and their frequency
D_congress_freqdist.most_common(20)

[('status', 16397),
 ('today', 15047),
 ('trump', 12286),
 ('house', 12037),
 ('people', 10355),
 ('act', 9833),
 ('congress', 9826),
 ('work', 9755),
 ('us', 9017),
 ('health', 8654),
 ('must', 8431),
 ('president', 8321),
 ('women', 7843),
 ('proud', 7806),
 ('need', 7640),
 ('time', 7601),
 ('bill', 7522),
 ('american', 7360),
 ('news', 7226),
 ('families', 7209)]

In [32]:
R_congress_freqdist =  nltk.FreqDist(Parties_Tweet["tweet"][1])

In [33]:
#showing Republicans most common word and their frequency
R_congress_freqdist.most_common(20)

[('today', 8326),
 ('status', 7619),
 ('house', 6071),
 ('great', 5836),
 ('news', 5751),
 ('border', 5041),
 ('work', 4712),
 ('senate', 4458),
 ('congress', 4267),
 ('bill', 4182),
 ('us', 3981),
 ('act', 3929),
 ('thank', 3917),
 ('people', 3886),
 ('new', 3821),
 ('bit', 3711),
 ('day', 3543),
 ('time', 3535),
 ('support', 3442),
 ('state', 3439)]

In [34]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [35]:
D_finder = BigramCollocationFinder.from_words(Parties_Tweet["tweet"][0])

In [36]:
D_scored = D_finder.score_ngrams(bigram_measures.raw_freq)

In [37]:
#showing democrats most common bi-grams words and their ratio to total 
D_scored[:20]

[(('health', 'care'), 0.0015831025332124478),
 (('climate', 'change'), 0.0010685114116687573),
 (('american', 'people'), 0.0009641856171160542),
 (('press', 'releases'), 0.0009203025448042028),
 (('trump', 'administration'), 0.0008855272799533017),
 (('president', 'trump'), 0.0008420581988896755),
 (('gun', 'violence'), 0.0008308804351876001),
 (('house', 'media'), 0.0006354765660253943),
 (('across', 'country'), 0.0006106370911318935),
 (('make', 'sure'), 0.0005882815637277428),
 (('pre', 'existing'), 0.0005675820013164921),
 (('federal', 'workers'), 0.0005398445876854163),
 (('town', 'hall'), 0.000527838841486891),
 (('look', 'forward'), 0.0005034133578416153),
 (('government', 'shutdown'), 0.00049554752412534),
 (('existing', 'conditions'), 0.00048519774291971466),
 (('every', 'day'), 0.0004740199792176393),
 (('united', 'states'), 0.00046118625052266396),
 (('years', 'ago'), 0.0004537344080546137),
 (('high', 'school'), 0.00043469081063626314)]

In [38]:
R_finder = BigramCollocationFinder.from_words(Parties_Tweet["tweet"][1])

In [39]:
#showing Republican most common bi-grams words and their ratio to total 
R_scored = R_finder.score_ngrams(bigram_measures.raw_freq)
R_scored[:20]

[(('press', 'releases'), 0.001300671850798119),
 (('border', 'security'), 0.0008297519094449539),
 (('senate', 'public'), 0.0008169633487372801),
 (('public', 'index'), 0.0008026702514757623),
 (('look', 'forward'), 0.0007846158128296346),
 (('house', 'media'), 0.0007229298141220315),
 (('southern', 'border'), 0.0007011140340912938),
 (('health', 'care'), 0.0006777937175067121),
 (('men', 'women'), 0.0006740323761221022),
 (('united', 'states'), 0.0006612438154144283),
 (('american', 'people'), 0.0006130986456914211),
 (('law', 'enforcement'), 0.0005973010118760593),
 (('high', 'school'), 0.0005784943049530096),
 (('president', 'trump'), 0.0005724761587376336),
 (('looking', 'forward'), 0.000479194892399307),
 (('last', 'week'), 0.00047016767307624314),
 (('national', 'security'), 0.0004664063316916332),
 (('youtube', 'watch'), 0.00045211323443011543),
 (('news', 'local'), 0.0004265361130147678),
 (('media', 'center'), 0.00040321579643018613)]

In [42]:
Dgram3 = ngrams(Parties_Tweet["tweet"][0], 3)
Rgram3 = ngrams(Parties_Tweet["tweet"][1], 3)

In [43]:
Dn3 = nltk.FreqDist(Dgram3)

In [44]:
#showing democrats most common tri-grams words and their total count 
Dn3.most_common(10)

[(('pre', 'existing', 'conditions'), 1172),
 (('media', 'press', 'releases'), 842),
 (('house', 'media', 'press'), 840),
 (('house', 'media', 'center'), 687),
 (('media', 'center', 'press'), 643),
 (('center', 'press', 'releases'), 643),
 (('nytimes', 'us', 'politics'), 619),
 (('congressional', 'art', 'competition'), 419),
 (('affordable', 'care', 'act'), 375),
 (('american', 'people', 'deserve'), 373)]

In [45]:
Rn3 = nltk.FreqDist(Rgram3)

In [46]:
#showing Republicans most common tri-grams words and their total count 
Rn3.most_common(10)

[(('senate', 'public', 'index'), 1066),
 (('house', 'media', 'center'), 499),
 (('media', 'center', 'press'), 487),
 (('center', 'press', 'releases'), 487),
 (('source', 'ig', 'share'), 434),
 (('ig', 'share', 'igshid'), 432),
 (('media', 'press', 'releases'), 432),
 (('documentsingle', 'aspx', 'documentid'), 428),
 (('house', 'media', 'press'), 422),
 (('house', 'news', 'documentsingle'), 402)]

In [47]:
Dgram4 = ngrams(Parties_Tweet["tweet"][0], 4)
Rgram4 = ngrams(Parties_Tweet["tweet"][1], 4)
Dn4 = nltk.FreqDist(Dgram4)
Rn4 = nltk.FreqDist(Rgram4)

In [48]:
#showing Democrats most common four-grams words and their total count
Dn4.most_common(10)

[(('house', 'media', 'press', 'releases'), 840),
 (('house', 'media', 'center', 'press'), 643),
 (('media', 'center', 'press', 'releases'), 643),
 (('people', 'pre', 'existing', 'conditions'), 322),
 (('house', 'news', 'documentsingle', 'aspx'), 317),
 (('news', 'documentsingle', 'aspx', 'documentid'), 316),
 (('martin', 'luther', 'king', 'jr'), 268),
 (('senate', 'news', 'press', 'releases'), 245),
 (('americans', 'pre', 'existing', 'conditions'), 240),
 (('equal', 'pay', 'equal', 'work'), 211)]

In [49]:
#showing Republicans most common four-grams words and their total count 
Rn4.most_common(10)

[(('media', 'center', 'press', 'releases'), 487),
 (('house', 'media', 'center', 'press'), 454),
 (('source', 'ig', 'share', 'igshid'), 432),
 (('house', 'media', 'press', 'releases'), 421),
 (('house', 'news', 'documentsingle', 'aspx'), 402),
 (('news', 'documentsingle', 'aspx', 'documentid'), 400),
 (('index', 'press', 'releases', 'id'), 371),
 (('senate', 'public', 'index', 'press'), 370),
 (('public', 'index', 'press', 'releases'), 370),
 (('born', 'alive', 'abortion', 'survivors'), 232)]

In [51]:
Dgram5 = ngrams(Parties_Tweet["tweet"][0], 5)
Rgram5 = ngrams(Parties_Tweet["tweet"][1], 5)
Dn5 = nltk.FreqDist(Dgram5)
Rn5 = nltk.FreqDist(Rgram5)

In [52]:
#showing Democrats most common five-grams words and their total count
Dn5.most_common(10)

[(('house', 'media', 'center', 'press', 'releases'), 643),
 (('house', 'news', 'documentsingle', 'aspx', 'documentid'), 316),
 (('dr', 'martin', 'luther', 'king', 'jr'), 176),
 (('house', 'media', 'press', 'releases', 'rep'), 165),
 (('duckworth', 'senate', 'news', 'press', 'releases'), 124),
 (('murphy', 'senate', 'newsroom', 'press', 'releases'), 116),
 (('click', 'module', 'top', 'stories', 'pgtype'), 112),
 (('module', 'top', 'stories', 'pgtype', 'homepage'), 112),
 (('action', 'click', 'module', 'top', 'stories'), 111),
 (('media', 'center', 'press', 'releases', 'congressman'), 110)]

In [53]:
#showing Republicans most common five-grams words and their total count 
Rn5.most_common(10)

[(('house', 'media', 'center', 'press', 'releases'), 454),
 (('house', 'news', 'documentsingle', 'aspx', 'documentid'), 400),
 (('senate', 'public', 'index', 'press', 'releases'), 370),
 (('public', 'index', 'press', 'releases', 'id'), 366),
 (('born', 'alive', 'abortion', 'survivors', 'protection'), 222),
 (('alive', 'abortion', 'survivors', 'protection', 'act'), 221),
 (('rubio', 'senate', 'public', 'index', 'press'), 220),
 (('senate', 'public', 'index', 'pressreleases', 'id'), 122),
 (('young', 'senate', 'newsroom', 'press', 'releases'), 104),
 (('wicker', 'senate', 'public', 'index', 'wicker'), 103)]

#### Stemming

In [54]:
porter = PorterStemmer()

In [57]:
Parties_Tweet['tweet'][0] = [porter.stem(word) for word in Parties_Tweet['tweet'][0]]
Parties_Tweet['tweet'][1] = [porter.stem(word) for word in Parties_Tweet['tweet'][1]]


In [58]:
#checking the size of new vocabulry after stemming
vocab5 = list(set(Parties_Tweet['tweet'][0]))
vocab6 = list(set(Parties_Tweet['tweet'][1]))
vocab_total_after_stem =list(set(vocab5+vocab6))
print (len(vocab5), len(vocab6))
print (len(vocab_total_after_stem))

108551 80894
163668


In [59]:
#cheking frequency of words
D_stem_count = nltk.FreqDist(Parties_Tweet["tweet"][0])
R_stem_count = nltk.FreqDist(Parties_Tweet["tweet"][1])

In [60]:
#we can see that most of the words are in very low frequency
D_stem_count.most_common()[20000:20010]

[('plumber', 3),
 ('ggi', 3),
 ('emce', 3),
 ('wink', 3),
 ('maryam', 3),
 ('fop', 3),
 ('wss', 3),
 ('incomprehens', 3),
 ('rusi', 3),
 ('fcdb', 3)]

In [2]:
D_stem_count.items()

#### Creating list of final features

In [96]:
#I would like to have no more than 10,000 features in my model,
#so I chose only words which appears more than 14 times in the entire corpus
Final_vocab_D = [word for word, freq in D_stem_count.items() if freq>=15] 
Final_vocab_R = [word for word, freq in R_stem_count.items() if freq>=15] 

In [97]:
vocab_total_D_R =list(set(Final_vocab_D + Final_vocab_R))
len(vocab_total_D_R)

9639

In [ ]:
#exporting the final list to json file
with open('vocab_total_D_R.json', 'w') as outfile:
     json.dump(vocab_total_D_R, outfile)